<a href="https://colab.research.google.com/github/Sakiba1705/CLIP-Vision-Transformer-Approach-Melanoma-Detection/blob/main/Melanoma_skin_cancer_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import vit_keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


Define data paths

In [ ]:
train_data_dir = 'train'
validation_data_dir = 'validation'
test_data_dir = 'test'

Define parameters
 

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
NUM_CLASSES = 2

Define data augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

Define data generators

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

In [ ]:
!pip install --upgrade vit-keras

Build ViT model

In [ ]:
def build_vit_model():
    inputs = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    vit_model = vit_keras.vit.Vit(
        include_top=False,
        input_tensor=inputs,
        classes=NUM_CLASSES
    )
    x = layers.Flatten()(vit_model(inputs))
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)
    return tf.keras.Model(inputs, outputs)


Call build_vit_model() to create the model

In [ ]:
vit_model = build_vit_model()


Compile and train the model

In [ ]:
vit_model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

history = vit_model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)

Evaluate the model
 

In [ ]:
test_loss, test_acc = vit_model.evaluate(test_generator)
print('Test accuracy:', test_acc)